In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [3]:
%%sql
CREATE TABLE DataFlowDiagrams
(diagram_name CHAR(10) NOT NULL,
 bubble_name CHAR(10) NOT NULL,
 flow_name CHAR(10) NOT NULL,
    PRIMARY KEY (diagram_name, bubble_name, flow_name));

INSERT INTO DataFlowDiagrams VALUES('Proc1',  'input' ,   'guesses');
INSERT INTO DataFlowDiagrams VALUES('Proc1',  'input' ,   'opinions'); 
INSERT INTO DataFlowDiagrams VALUES('Proc1',  'crunch',   'facts');
INSERT INTO DataFlowDiagrams VALUES('Proc1',  'crunch',   'guesses');
INSERT INTO DataFlowDiagrams VALUES('Proc1',  'crunch',   'opinions' );
INSERT INTO DataFlowDiagrams VALUES('Proc1',  'output',   'facts');
INSERT INTO DataFlowDiagrams VALUES('Proc1',  'output',   'guesses');
INSERT INTO DataFlowDiagrams VALUES('Proc2',  'reckon',   'guesses');
INSERT INTO DataFlowDiagrams VALUES('Proc2',  'reckon',   'opinions' );

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [51]:
%%sql
-- 全ての候補を列挙するクエリ
select distinct d1.diagram_name, d1.bubble_name, d2.flow_name
    from DataFlowDiagrams as d1, DataFlowDiagrams as d2

*  postgresql://padawan:***@db:5432/dsdojo_db
12 rows affected.


diagram_name,bubble_name,flow_name
Proc1,input,guesses
Proc1,input,opinions
Proc2,reckon,opinions
Proc1,crunch,facts
Proc2,reckon,facts
Proc2,reckon,guesses
Proc1,input,facts
Proc1,output,guesses
Proc1,output,facts
Proc1,crunch,opinions


In [53]:
%%sql
-- 全ての候補からnot existsでfilterする
select distinct d1.diagram_name, d1.bubble_name, d2.flow_name
from DataFlowDiagrams as d1, DataFlowDiagrams as d2
where not exists (
    select *
    from DataFlowDiagrams as d3
    where d1.diagram_name = d3.diagram_name and
    d1.bubble_name = d3.bubble_name and
    d2.flow_name = d3.flow_name
)

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


diagram_name,bubble_name,flow_name
Proc2,reckon,facts
Proc1,input,facts
Proc1,output,opinions


In [56]:
%%sql
-- 全ての候補からexceptでフィルタする
select distinct d1.diagram_name, d1.bubble_name, d2.flow_name
from DataFlowDiagrams as d1, DataFlowDiagrams as d2
except
select *
from DataFlowDiagrams

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


diagram_name,bubble_name,flow_name
Proc2,reckon,facts
Proc1,input,facts
Proc1,output,opinions


In [8]:
%%sql
select distinct d1.diagram_name, d1.bubble_name, d2.flow_name
from DataFlowDiagrams as d1
cross join DataFlowDiagrams as d2
-- left joinとNULL判定の組み合わせ
left join DataFlowDiagrams as d3
on d1.diagram_name = d3.diagram_name and
d1.bubble_name = d3.bubble_name and
d2.flow_name = d3.flow_name
where d3.flow_name is NULL

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


diagram_name,bubble_name,flow_name
Proc2,reckon,facts
Proc1,input,facts
Proc1,output,opinions
